## Build a Question Answer Application over a Graph Database

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
NEO4J_URI = os.environ["NEO4J_URI"]
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.environ["NEO4J_PASSWORD"]

In [ ]:
import os
from langchain_community.graphs import Neo4jGraph
from dotenv import load_dotenv

load_dotenv()

try:
    graph = Neo4jGraph(
        url=os.getenv("NEO4J_URI"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD"),
        database=os.getenv("NEO4J_DATABASE") or "neo4j",
    )
    print("Neo4j Graph initialized successfully!")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
## Dataset Moview 
movie_query="""

LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""


In [ ]:
movie_query

In [ ]:
graph.query(movie_query)

In [ ]:
graph.refresh_schema()

In [ ]:
graph.schema